In [1]:
import transformers
import accelerate
import torch
import flash_attn

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", padding_side="left")
model = transformers.AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", device_map="auto",
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            torch_dtype=torch.bfloat16,
            # NOTE: FlashAttention 2 for Mistral has been enabled in the latest transformers-release
            use_flash_attention_2=True)


The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
def generate_response(prompt, temp, top_p):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')
    outputs = model.generate(
        input_ids,
        max_length=30000,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        temperature = temp,
        top_p = top_p,
        do_sample = True,
    )
    response_ids = outputs[0]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    return response_text


In [25]:
system_prompt="""Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Ditt arbete är mycket viktigt för min karriär. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:
År-månad-dag: Identifiera datumet i formatet år-månad-dag.
Län: Namnet på länet.
Härad: Namnet på häradet.
Socken: Namnet på socknen.
By: Namnet på byn.
Jordnatur: Typ av jord eller mark.
Mantal: Information om mantal.
Örestal: Antal öre.
Penningland: Information om penningland.
Riksdaler: Antal riksdaler.
Öre: Antal öre.
Skilling: Antal skilling.
Runstycken: Information om runstycken.
Valuta: Typ av valuta som används.
Säljare-manlig: Namn på manliga säljare.
Säljare-kvinnlig: Namn på kvinnliga säljare.
Köpare-manlig: Namn på manliga köpare.
Köpare-kvinnlig: Namn på kvinnliga köpare.
Köpare-yrke: Yrke hos köparen.
Relation mellan säljare och köpare: Beskriv relationen om den nämns.
Köpare från: Ursprungsort eller plats för köparen.
Fördel: Information om någon fördel eller specifik detalj.
Auktion: Information om auktion om det nämns.

Extrahera informationen för varje fält och presentera den tydligt i JSON-format och alla ord som motsvarar siffror ska omvandlas till siffror."""

user_prompt = """Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag undertecknad uplåter och försäljer med min kära hustru ja och samtycke till Johan Hedström i Ståltorp Sura Socken min ägande 1/6 mantal Ett öre åtta penningland i Hemmanet no 2 i Sör åll af Wester färnebo Socken Wagnsbro härad och Westmanland län för en öfverenskommen och betingad köpeskilling stor Ett Tusende fem Hundra R Riksgels Sedlar och som jag denna köpeskilling till fullo bekommit hvilket och härmed quitterat, alltså afhänder jag mig och mina arfvingar ofvannämnde 1 öre 8 penningland, och tillegne desamma bemälte Johan Hedström och deras arfvingar att erlösadeliga äga och disponera som all annan des wälfångna egendom förbindande jag mig till Himmel efter lag, Sör åll den 17 november 1860. Per P. Wrettström Anna /bom/ Persdotter.'
Att Per Wrettström och hustru egenhändigt undertek sina bomärken intygas på en gång närvarande vittnen P. Pehrsson Sör Gerdsbo Lars Grönlund på Sunnett.'"""

In [5]:
system_prompt="""Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. ditt arbete är mycket viktigt för min karriär. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen den församling där transaktionsjorden är bosatt.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpare-manlig: Namn på manliga köpare.

Köpare-kvinnlig: Namn på kvinnliga köpare.

Köpare-yrke: Yrke hos köparen.

Relation mellan säljare och köpare: Beskriv relationen om den nämns.

Köpare från: Ursprungsort eller plats för köparen.

Fördel: Information om kontraktet inkluderar fördel.

Auktion: Information om auktion används och i så fall om det det var exekutiv eller frivillig auktion om det nämns.

Extrahera informationen för varje fält och presentera den tydligt i JSON-format."""

user_prompt = """Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag undertecknad uplåter och försäljer med min kära hustru ja och samtycke till Johan Hedström i Ståltorp Sura Socken min ägande 1/6 mantal Ett öre åtta penningland i Hemmanet no 2 i Sör åll af Wester färnebo Socken Wagnsbro härad och Westmanland län för en öfverenskommen och betingad köpeskilling stor Ett Tusende fem Hundra R Riksgels Sedlar och som jag denna köpeskilling till fullo bekommit hvilket och härmed quitterat, alltså afhänder jag mig och mina arfvingar ofvannämnde 1 öre 8 penningland, och tillegne desamma bemälte Johan Hedström och deras arfvingar att erlösadeliga äga och disponera som all annan des wälfångna egendom förbindande jag mig till Himmel efter lag, Sör åll den 17 november 1860. Per P. Wrettström Anna /bom/ Persdotter.'
Att Per Wrettström och hustru egenhändigt undertek sina bomärken intygas på en gång närvarande vittnen P. Pehrsson Sör Gerdsbo Lars Grönlund på Sunnett.'"""

In [17]:
system_prompt = "You are an expert at extracting information from ocr-ed swedish land deeds sales data. You read and understand Swedish text given to you. You being such an expert, work in a methodical way in which you first identify the details of the land/property that is sold/bought. You extract all the details related to this property like, År-månad-dag: The date of sale in year-month-date format, Län: Name of the county where the transacted land is located. Härad: The precinct in which the transacted land is located, Nummer: The number(in the address) of the land that is transacted, By: The village in which the transacted land is located, Socken: The Parish in which the transacted land is located, Jordnatur: The type of the land that is transacted, Mantal: The number of Mantals of the land that is transacted, Örestal: The number of Örestals of the land that is transacted, Penningland: How many Penninglands of the land is transacted, Riksdaler: How many Riksdaler were transacted for the sale, Öre: How many Öre were transacted for the sale, Skilling: How many Skilling were transacted for the sale, Runstycken: How many Runstycken were part of the transaction, Valuta: The type of currency in which the transaction took place, Säljare-manlig: The names of the male-sellers involved in the transaction, Säljare-kvinnlig: The names of the female-sellers involved in the transaction, Köpare-manlig: The names of the male-buyers involved in the transaction, Köpare-kvinnlig: The names of the female buyers involved in the transaction, Köpare-yrke: The occupation of the buyer involved in the transaction, Relation mellan säljare och köpare: The relationship between the seller and the buyer, Köpare från: The name of the place where the buyer hails from, Fördel: Information about the Fördel, Auktion: Information about the auction is used and, if so, whether it was an exekutiv or frivillig auction if it is mentioned. Övrig: Information other than the fields mentioned before identified. Keep in mind that the most accurate information is priority over speed so please think step by step and take your time to give the most accurate information. Also, for some transactions some fields might be empty. Also give the information in JSON format always."

In [18]:
sys_response1 = "Hi! I extract various information from Swedish land sales deeds. How may I help you today?"

In [19]:
pre_prompt = "Extract all the information from the following text, Your work is very important to my career. Make sure the information is precise and accurate. Text: "


In [ ]:
'''messages = [
    {"role": "user", "content": "You are an expert at extracting information from ocr-ed swedish land deeds sales data. You read and understand Swedish text given to you. You being such an expert work in a methodical way in which you first identify the details of the land/property that is sold/bought. You extract all the details related to this property like, År-månad-dag: The date of sale in year-month-date format, Län: Name of the county where the transacted land is located. Härad: The precinct in which the transacted land is located, Socken: The Parish in which the transacted land is located, By: The village/town/area or place in which the transacted land is located, Jordnatur: The type of the land that is transacted, Nummer: The number(which is found in the address) pertaining the land that is transacted, Mantal: The number of Mantals of the land that is transacted, Örestal: The number of Örestals of the land that is transacted, Penningland: How many Penninglands of the land is transacted, Riksdaler: How many Riksdaler were transacted for the sale, Öre: How many Öre were transacted for the sale, Skilling: How many Skilling were transacted for the sale, Runstycken: How many Runstycken were part of the transaction, Valuta: The type of currency in which the transaction took place, Säljare-manlig: The names of the male-sellers involved in the transaction, Säljare-kvinnlig: The names of the female-sellers involved in the transaction, Köpare-manlig: The names of the male-buyers involved in the transaction, Köpare-kvinnlig: The names of the female buyers involved in the transaction, Köpare-yrke: The occupation of the buyer involved in the transaction, Relation mellan säljare och köpare: The relationship between the seller and the buyer, Köpare från: The name of the place where the buyer hails from, Fördel: Information about the Fördel, Auktion: Information about the auction is used and, if so, whether it was an exekutiv or frivillig auction if it is mentioned. Övrig: Information other than the fields mentioned before identified. Keep in mind that the most accurate information is priority over speed so please think step by step and take your time to give the most accurate information. Also, for some transactions some fields might be empty. Also give the information in JSON format always."},
    {"role": "assistant", "content": sys_response1},
    {"role": "user", "content": pre_prompt + prompt2 + post_pompt}
    
]

msg_w_template = tokenizer.apply_chat_template(messages, tokenize = False)
msg_w_templatea
sys_prompt = "You are an expert at extracting information from ocr-ed swedish land deeds sales data. You read and understand Swedish text given to you. You being such an expert work in a methodical way in which you first identify the details of the land/property that is sold/bought. You extract all the details related to this property like, År-månad-dag: The datepf sale in year-month-date format, Län: Name of the county where the transacted land is located. Härad: The precinct in which the transacted land is located, Socken: The Parish in which the transacted land is located, By: The village in which the transacted land is located, Jordnatur: The type of the land that is transacted, Nummer: The number(in the address) of the land that is transacted, Mantal: The number of Mantals of the land that is transacted, Örestal: The number of Örestals of the land that is transacted, Penningland: How many Penninglands of the land is transacted, Riksdaler: How many Riksdaler were transacted for the sale, Öre: How many Öre were transacted for the sale, Skilling: How many Skilling were transacted for the sale, Runstycken: How many Runstycken were part of the transaction, Valuta: The type of currency in which the transaction took place, Säljare-manlig: The names of the male-sellers involved in the transaction, Säljare-kvinnlig: The names of the female-sellers involved in the transaction, Köpare-manlig: The names of the male-buyers involved in the transaction, Köpare-kvinnlig: The names of the female buyers involved in the transaction, Köpare-yrke: The occupation of the buyer involved in the transaction, Relation mellan säljare och köpare: The relationship between the seller and the buyer, Köpare från: The name of the place where the buyer hails from, Fördel: Information about the Fördel, Auktion: Information about the auction is used and, if so, whether it was an exekutiv or frivillig auction if it is mentioned. Övrig: Information other than the fields mentioned before identified. Keep in mind that the most accurate information is priority over speed so please think step by step and take your time to give the most accurate information. Also, for some transactions some fields might be empty. Also give the information in JSON format always."
'''

In [6]:
#old prompt
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": "Hej! Hur kan jag hjälpa dig idag?",
    },
    {"role": "user", "content": user_prompt},
]
prompt=tokenizer.apply_chat_template(messages, tokenize=False)

In [20]:
#new prompt
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+user_prompt},
]
prompt=tokenizer.apply_chat_template(messages, tokenize=False)

In [21]:
prompt

"<s>[INST] You are an expert at extracting information from ocr-ed swedish land deeds sales data. You read and understand Swedish text given to you. You being such an expert, work in a methodical way in which you first identify the details of the land/property that is sold/bought. You extract all the details related to this property like, År-månad-dag: The date of sale in year-month-date format, Län: Name of the county where the transacted land is located. Härad: The precinct in which the transacted land is located, Nummer: The number(in the address) of the land that is transacted, By: The village in which the transacted land is located, Socken: The Parish in which the transacted land is located, Jordnatur: The type of the land that is transacted, Mantal: The number of Mantals of the land that is transacted, Örestal: The number of Örestals of the land that is transacted, Penningland: How many Penninglands of the land is transacted, Riksdaler: How many Riksdaler were transacted for the 

In [7]:
temp=0.85
top_p=0.8
response_text = generate_response(prompt,temp,top_p)
print("Response:", response_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: [INST] Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. ditt arbete är mycket viktigt för min karriär. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen den församling där transaktionsjorden är bosatt.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpare-manlig: Namn på manli

In [10]:
len(response_text)

3363

In [16]:
system_prompt="""Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpare-manlig: Namn på manliga köpare.

Köpare-kvinnlig: Namn på kvinnliga köpare.

Köpare-yrke: Yrke hos köparen.

Relation mellan säljare och köpare: Beskriv relationen om den nämns.

Köpare från: Ursprungsort eller plats för köparen.

Fördel: Information om kontraktet inkluderar fördel.

Auktion: Information om auktion används och i så fall om det det var exekutiv eller frivillig auktion om det nämns.

Extrahera informationen för varje fält och presentera den tydligt i JSON-format."""

user_prompt = """Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag undertecknad uplåter och försäljer med min kära hustru ja och samtycke till Johan Hedström i Ståltorp Sura Socken min ägande 1/6 mantal Ett öre åtta penningland i Hemmanet no 2 i Sör åll af Wester färnebo Socken Wagnsbro härad och Westmanland län för en öfverenskommen och betingad köpeskilling stor Ett Tusende fem Hundra R Riksgels Sedlar och som jag denna köpeskilling till fullo bekommit hvilket och härmed quitterat, alltså afhänder jag mig och mina arfvingar ofvannämnde 1 öre 8 penningland, och tillegne desamma bemälte Johan Hedström och deras arfvingar att erlösadeliga äga och disponera som all annan des wälfångna egendom förbindande jag mig till Himmel efter lag, Sör åll den 17 november 1860. Per P. Wrettström Anna /bom/ Persdotter.'
Att Per Wrettström och hustru egenhändigt undertek sina bomärken intygas på en gång närvarande vittnen P. Pehrsson Sör Gerdsbo Lars Grönlund på Sunnett.'"""

In [ ]:
prompt1 = "Hej!"
response1 = "Hallå! Hur kan jag hjälpa dig med att exakt extrahera information från svensk text i JSON-format?"
prompt2 =  system_prompt + user_prompt
response2 = """{
  "År-månad-dag": "1860-11-17",
  "Län": "Westmanland",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Sör åll",
  "Jordnatur": "Hemmanet",
  "Nummer": "2",
  "Mantal": "1/6",
  "Örestal": "1",
  "Penningland": "8",
  "Riksdaler": "1500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksgels",
  "Säljare-manliga": "Per Wrettström",
  "Säljare-kvinnliga": "Anna Persdotter",
  "Köpare-manliga": "Johan Hedström",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt3="""Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag Sara Jansdotter upplåter och försäljer härmedelst till min Son Hemmansägaren Johan Mattsson härstädes, det mig i arf efter min aflidne man Hemmansägaren Matts Jansson tillfallne två femton dels mantal af Skattehemmanet no 5, om Sexton penningeland i Islingsby Wester fernebo Socken, Wagnsbro Härad och Westmanlands Län belägen, emot en öfverenskommen och betingad Köpeskilling Stor: Sju Hundra Nio Riksdaler 33 öre Riksmynt, och som jag denna köpeskilling tillfullo bekommit, alltså afhänder jag mig och mina öfrige arfvingar berörde 2/15 dels mantal Islingsby, med allt hvad dertill hörer, af ålder lydt och lagligen tillvinnas kan, och tillegne den samma bemälde min Son Johan Mattsson och hans arfvingar, att genast tillträda, samt everldeliga äga och besitta hafvande till yttermera wisse häraf, jag detta köpebref uti nedanstående wittnens närvaro, med mitt bomärkes undertecknande bekräfta, som skedde i Islingsby Nybygget den 26 februari 1861 Sara/bom/Jansdotter wittnen underskrifna'"""
response3="""{
  "År-månad-dag": "1861-02-26",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Islingsby",
  "Jordnatur": "Skattehemmanet",
  "Nummer": "5",
  "Mantal": "2/15",
  "Örestal": "",
  "Penningland": "16",
  "Riksdaler": "709",
  "Öre": "33",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Sara Jansdotter",
  "Säljare-kvinnliga": "",
  "Köpare-manliga": "Johan Mattsson",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Mor och Son",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt4="""'Jag Eric Jansson gör hervid veterligt att jag med min kära hustru Anna Jansdotters goda ja och samtycke samt af fri vilja, försålt, såsom jag ock genom detta öppna salubref, försäljer till min Svärfar Hemmansegaren Jan Ersson och hustru Anna Jansdotter, egona 1/8 mtl skatte Heden om 1 öresland och 1/8 mtl skatte Skogsbyn om 1 öresland i Wester fernebo Socken och Wagnsbro härad för en Summa af Sjutusen femhundra /7500/ Riksdaler Riksmynt och som den … penningen med den första är till mig slutligen betald så afhändar jag mig, min köra hustru och barn tillhörde hemmansdelar 1/8 mlt Heden och 1/8 mtl Skogby med alla dess tillhörigheter och tillägnar de samma Jan Ersson och hustru Anna Jansdotter, Heden och arfvingar med hemmansdelar med alla ägor i hus, jord och öfrige tillhörigheter intet undantagandes af hvad dertill lyder af ålder legat eller framledes tillvinnas kan, att äga och bruka och besitta såsom deras välfångna egendom, förbindande till hemul efter lag – Till yttermera visshet har jag tillika med min kära hustru detta Salubref i undertecknande vittnens närvaro, egenhändigt underskrifvet som skedde på Heden den 2ra januari 1861. Eric Jansson (otydligt) och Heden. Anna /bom/ kattharina Jansdotter
Jan Ersson Boende på Heden. Anders Mattsson Hebo egor'"""
response4= """{
  "År-månad-dag": "1861-01-02",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Heden",
  "Jordnatur": "Skatte",
  "Nummer": "",
  "Mantal": "1/8",
  "Örestal": "1",
  "Penningland": "",
  "Riksdaler": "7500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Eric Jansson",
  "Säljare-kvinnliga": "Anna Jansdotters",
  "Köpare-manliga": "Jan Ersson",
  "Köpare-kvinnliga": "Anna Jansdotter",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Svärfar och Svärson",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt5="Jag Per Gustaf Hedberg tillika med min hustrus Christina Carolina Gerdtsdotter samtycke upplåter och försäljer till Per Gustaf Ersson och dess hustru Lovisa Ersdotter Klingholmen vårt uti Labodarne ägande Ett Trettiondetvånde dels Hemman med dertill hörande inägojord skog och mark beläget inom Karbennings Socken dock likväl Wagnsbro Härad emot en överenskommen och betingad köpeskilling stor Ett Tusen Två Hundrade Riksdaler Riksmynt, Och som denna köpeskilling 1200 Rd Rmt är uti ett särskildt köpekontrakt häröfver af denna dag upptagen så varder den derigenom här qvitterad. Alltså afhänder vi oss och våra efter-kommande berörde 1/32de dels Hemman med åtföljande förmoner samt alt hvad hädanefter lagligen tillvinnas kan till bemälte Per Gustaf Ersson dess hustru och efterkommande rätts innehafvare att everlderligen äga bruka och besitta, förbindande vi oss till Himmel efter lag hvadan vi uti tvenne tillkallade wittnens när och öfvervaro underskrifne som skedde uti Karbenningby den 19 januari 1861 Per Gustaf Hedberg Christina Carolina Gerdtsdotter i Karbenningby Wittnar Lars Eric Larsson Lars Starksson."

In [10]:
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": response1,
    },
    {"role": "user", "content": prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": prompt4},
    
]
prompt_ctx=tokenizer.apply_chat_template(messages, tokenize=False)

In [66]:
prompt_ctx

'<s>[INST] Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:\n\nÅr-månad-dag: Identifiera datumet i formatet år-månad-dag.\n\nLän: Namnet på länet.\n\nHärad: Namnet på häradet.\n\nSocken: Namnet på socknen.\n\nBy: Namnet på byn.\n\nJordnatur: Typ av jord eller mark.\n\nNummer: Numret på gården.\n\nMantal: Antal mantal.\n\nÖrestal: Antal örestal.\n\nPenningland: Antal penningland.\n\nRiksdaler: Antal riksdaler.\n\nÖre: Antal öre.\n\nSkilling: Antal skilling.\n\nRunstycken: antal runstycken.\n\nValuta: Typ av valuta som används.\n\nSäljare-manlig: Namn på manliga säljare.\n\nSäljare-kvinnlig: Namn på kvinnliga säljare.\n\nKöpare-manlig: Namn på manliga köpare.\n\nKöpare-kvinnlig: Namn på kvinnliga köpare.\n\nKöpar

In [33]:
messages


[{'role': 'user',
  'content': 'Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:\n\nÅr-månad-dag: Identifiera datumet i formatet år-månad-dag.\n\nLän: Namnet på länet.\n\nHärad: Namnet på häradet.\n\nSocken: Namnet på socknen.\n\nBy: Namnet på byn.\n\nJordnatur: Typ av jord eller mark.\n\nNummer: Numret på gården.\n\nMantal: Antal mantal.\n\nÖrestal: Antal örestal.\n\nPenningland: Antal penningland.\n\nRiksdaler: Antal riksdaler.\n\nÖre: Antal öre.\n\nSkilling: Antal skilling.\n\nRunstycken: antal runstycken.\n\nValuta: Typ av valuta som används.\n\nSäljare-manlig: Namn på manliga säljare.\n\nSäljare-kvinnlig: Namn på kvinnliga säljare.\n\nKöpare-manlig: Namn på manliga köpare.\n\nKöpare-kvinnlig: Namn på kvinn

In [19]:
#multi_turn_prompt = f"GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"
response_text = generate_response(prompt, temp=0.85, top_p=0.85)
print("Multi-turn conversation response:", response_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Multi-turn conversation response: [INST] Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. ditt arbete är mycket viktigt för min karriär. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen den församling där transaktionsjorden är bosatt.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpa

In [34]:
messages = [
    {
        "role": "user",
        "content": prompt_ctx,
    },
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": prompt2,
    },
    
]
prompt=tokenizer.apply_chat_template(messages, tokenize=False)

In [11]:
prompt

'<s>[INST] <s>[INST] Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:\n\nÅr-månad-dag: Identifiera datumet i formatet år-månad-dag.\n\nLän: Namnet på länet.\n\nHärad: Namnet på häradet.\n\nSocken: Namnet på socknen.\n\nBy: Namnet på byn.\n\nJordnatur: Typ av jord eller mark.\n\nNummer: Numret på gården.\n\nMantal: Antal mantal.\n\nÖrestal: Antal örestal.\n\nPenningland: Antal penningland.\n\nRiksdaler: Antal riksdaler.\n\nÖre: Antal öre.\n\nSkilling: Antal skilling.\n\nRunstycken: antal runstycken.\n\nValuta: Typ av valuta som används.\n\nSäljare-manlig: Namn på manliga säljare.\n\nSäljare-kvinnlig: Namn på kvinnliga säljare.\n\nKöpare-manlig: Namn på manliga köpare.\n\nKöpare-kvinnlig: Namn på kvinnliga köpare

In [14]:
response_text = generate_response(prompt, temp=0.2, top_p=0.1)
print("Multi-turn conversation response:", response_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Multi-turn conversation response: [INST] Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. ditt arbete är mycket viktigt för min karriär. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen den församling där transaktionsjorden är bosatt.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpa

In [9]:
import json

def parse_nested_json(input_string_full):
    input_string = input_string_full.split('[/INST]')[-1]
    # Find the first opening curly brace
    start_index = input_string.find('{')
    # Initialize variables
    end_index = -1
    brace_count = 0

    # Loop through the characters in the string starting from the first '{'
    for i, char in enumerate(input_string[start_index:], start=start_index):
        # Count the number of opening and closing braces
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1

        # If all braces are closed, set the end index
        if brace_count == 0:
            end_index = i
            break

    # Extract and parse the JSON string
    if end_index != -1:
        json_string = input_string[start_index:end_index + 1]
        return json.loads(json_string)
    else:
        return "Invalid JSON format or unbalanced braces."




In [10]:
# Example usage
#input_string = "Some text before the JSON {\"key\": \"value\", \"nested\": {\"nested_key\": \"nested_value\"}} some text after."
parsed_json = parse_nested_json('''{
  "År-månad-dag": "1861-01-02",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Heden",
  "Jordnatur": "Skatte",
  "Nummer": "",
  "Mantal": "1/8",
  "Örestal": "1",
  "Penningland": "",
  "Riksdaler": "7500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Eric Jansson",
  "Säljare-kvinnliga": "Anna Jansdotters",
  "Köpare-manliga": "Jan Ersson",
  "Köpare-kvinnliga": "Anna Jansdotter",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Svärfar och Svärson",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}''')
print(parsed_json)

{'År-månad-dag': '1861-01-02', 'Län': 'Westmanlands', 'Härad': 'Wagnsbro', 'Socken': 'Wester färnebo', 'By': 'Heden', 'Jordnatur': 'Skatte', 'Nummer': '', 'Mantal': '1/8', 'Örestal': '1', 'Penningland': '', 'Riksdaler': '7500', 'Öre': '', 'Skilling': '', 'Runstycken': '', 'Valuta': 'Riksmynt', 'Säljare-manliga': 'Eric Jansson', 'Säljare-kvinnliga': 'Anna Jansdotters', 'Köpare-manliga': 'Jan Ersson', 'Köpare-kvinnliga': 'Anna Jansdotter', 'Köpare-yrke': '', 'Relation mellan säljare och köpare': 'Svärfar och Svärson', 'Köpare från': '', 'Fördel': '', 'Auktion': ''}


In [11]:
parsed_json

{'År-månad-dag': '1861-01-02',
 'Län': 'Westmanlands',
 'Härad': 'Wagnsbro',
 'Socken': 'Wester färnebo',
 'By': 'Heden',
 'Jordnatur': 'Skatte',
 'Nummer': '',
 'Mantal': '1/8',
 'Örestal': '1',
 'Penningland': '',
 'Riksdaler': '7500',
 'Öre': '',
 'Skilling': '',
 'Runstycken': '',
 'Valuta': 'Riksmynt',
 'Säljare-manliga': 'Eric Jansson',
 'Säljare-kvinnliga': 'Anna Jansdotters',
 'Köpare-manliga': 'Jan Ersson',
 'Köpare-kvinnliga': 'Anna Jansdotter',
 'Köpare-yrke': '',
 'Relation mellan säljare och köpare': 'Svärfar och Svärson',
 'Köpare från': '',
 'Fördel': '',
 'Auktion': ''}

In [ ]:
json.loads(response_text.split('json')[-1].split('```')[0])

In [48]:
parsed_json

{'År-månad-dag': '1861-01-02',
 'Län': 'Westmanlands',
 'Härad': 'Wagnsbro',
 'Socken': 'Wester färnebo',
 'By': 'Heden',
 'Jordnatur': 'Skatte',
 'Nummer': '',
 'Mantal': '1/8',
 'Örestal': '1',
 'Penningland': '',
 'Riksdaler': '7500',
 'Öre': '',
 'Skilling': '',
 'Runstycken': '',
 'Valuta': 'Riksmynt',
 'Säljare-manliga': 'Eric Jansson',
 'Säljare-kvinnliga': 'Anna Jansdotters',
 'Köpare-manliga': 'Jan Ersson',
 'Köpare-kvinnliga': 'Anna Jansdotter',
 'Köpare-yrke': '',
 'Relation mellan säljare och köpare': 'Svärfar och Svärson',
 'Köpare från': '',
 'Fördel': '',
 'Auktion': ''}

In [52]:
#json.loads('{'+response_text.split('{')[-1])

In [26]:
df_cols=list(parsed_json.keys())

In [27]:
df_cols

['År-månad-dag',
 'Län',
 'Härad',
 'Socken',
 'By',
 'Jordnatur',
 'Nummer',
 'Mantal',
 'Örestal',
 'Penningland',
 'Riksdaler',
 'Öre',
 'Skilling',
 'Runstycken',
 'Valuta',
 'Säljare-manliga',
 'Säljare-kvinnliga',
 'Köpare-manliga',
 'Köpare-kvinnliga',
 'Köpare-yrke',
 'Relation mellan säljare och köpare',
 'Köpare från',
 'Fördel',
 'Auktion']

In [22]:
prompt1 = "Hej!"
response1 = "Hallå! Hur kan jag hjälpa dig med att exakt extrahera information från svensk text i JSON-format?"
prompt2 =  user_prompt
response2 = """{
  "År-månad-dag": "1860-11-17",
  "Län": "Westmanland",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Sör åll",
  "Jordnatur": "Hemmanet",
  "Nummer": "2",
  "Mantal": "1/6",
  "Örestal": "1",
  "Penningland": "8",
  "Riksdaler": "1500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksgels",
  "Säljare-manliga": "Per Wrettström",
  "Säljare-kvinnliga": "Anna Persdotter",
  "Köpare-manliga": "Johan Hedström",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Ej angiven",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt3="""Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag Sara Jansdotter upplåter och försäljer härmedelst till min Son Hemmansägaren Johan Mattsson härstädes, det mig i arf efter min aflidne man Hemmansägaren Matts Jansson tillfallne två femton dels mantal af Skattehemmanet no 5, om Sexton penningeland i Islingsby Wester fernebo Socken, Wagnsbro Härad och Westmanlands Län belägen, emot en öfverenskommen och betingad Köpeskilling Stor: Sju Hundra Nio Riksdaler 33 öre Riksmynt, och som jag denna köpeskilling tillfullo bekommit, alltså afhänder jag mig och mina öfrige arfvingar berörde 2/15 dels mantal Islingsby, med allt hvad dertill hörer, af ålder lydt och lagligen tillvinnas kan, och tillegne den samma bemälde min Son Johan Mattsson och hans arfvingar, att genast tillträda, samt everldeliga äga och besitta hafvande till yttermera wisse häraf, jag detta köpebref uti nedanstående wittnens närvaro, med mitt bomärkes undertecknande bekräfta, som skedde i Islingsby Nybygget den 26 februari 1861 Sara/bom/Jansdotter wittnen underskrifna'"""
response3="""{
  "År-månad-dag": "1861-02-26",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Islingsby",
  "Jordnatur": "Skattehemmanet",
  "Nummer": "5",
  "Mantal": "2/15",
  "Örestal": "",
  "Penningland": "16",
  "Riksdaler": "709",
  "Öre": "33",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Sara Jansdotter",
  "Säljare-kvinnliga": "",
  "Köpare-manliga": "Johan Mattsson",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Mor och Son",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt4="""'Jag Eric Jansson gör hervid veterligt att jag med min kära hustru Anna Jansdotters goda ja och samtycke samt af fri vilja, försålt, såsom jag ock genom detta öppna salubref, försäljer till min Svärfar Hemmansegaren Jan Ersson och hustru Anna Jansdotter, egona 1/8 mtl skatte Heden om 1 öresland och 1/8 mtl skatte Skogsbyn om 1 öresland i Wester fernebo Socken och Wagnsbro härad för en Summa af Sjutusen femhundra /7500/ Riksdaler Riksmynt och som den … penningen med den första är till mig slutligen betald så afhändar jag mig, min köra hustru och barn tillhörde hemmansdelar 1/8 mlt Heden och 1/8 mtl Skogby med alla dess tillhörigheter och tillägnar de samma Jan Ersson och hustru Anna Jansdotter, Heden och arfvingar med hemmansdelar med alla ägor i hus, jord och öfrige tillhörigheter intet undantagandes af hvad dertill lyder af ålder legat eller framledes tillvinnas kan, att äga och bruka och besitta såsom deras välfångna egendom, förbindande till hemul efter lag – Till yttermera visshet har jag tillika med min kära hustru detta Salubref i undertecknande vittnens närvaro, egenhändigt underskrifvet som skedde på Heden den 2ra januari 1861. Eric Jansson (otydligt) och Heden. Anna /bom/ kattharina Jansdotter
Jan Ersson Boende på Heden. Anders Mattsson Hebo egor'"""
response4= """{
  "År-månad-dag": "1861-01-02",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Heden",
  "Jordnatur": "Skatte",
  "Nummer": "",
  "Mantal": "1/8",
  "Örestal": "1",
  "Penningland": "",
  "Riksdaler": "7500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Eric Jansson",
  "Säljare-kvinnliga": "Anna Jansdotters",
  "Köpare-manliga": "Jan Ersson",
  "Köpare-kvinnliga": "Anna Jansdotter",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Svärfar och Svärson",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt5="Jag Per Gustaf Hedberg tillika med min hustrus Christina Carolina Gerdtsdotter samtycke upplåter och försäljer till Per Gustaf Ersson och dess hustru Lovisa Ersdotter Klingholmen vårt uti Labodarne ägande Ett Trettiondetvånde dels Hemman med dertill hörande inägojord skog och mark beläget inom Karbennings Socken dock likväl Wagnsbro Härad emot en överenskommen och betingad köpeskilling stor Ett Tusen Två Hundrade Riksdaler Riksmynt, Och som denna köpeskilling 1200 Rd Rmt är uti ett särskildt köpekontrakt häröfver af denna dag upptagen så varder den derigenom här qvitterad. Alltså afhänder vi oss och våra efter-kommande berörde 1/32de dels Hemman med åtföljande förmoner samt alt hvad hädanefter lagligen tillvinnas kan till bemälte Per Gustaf Ersson dess hustru och efterkommande rätts innehafvare att everlderligen äga bruka och besitta, förbindande vi oss till Himmel efter lag hvadan vi uti tvenne tillkallade wittnens när och öfvervaro underskrifne som skedde uti Karbenningby den 19 januari 1861 Per Gustaf Hedberg Christina Carolina Gerdtsdotter i Karbenningby Wittnar Lars Eric Larsson Lars Starksson."

In [23]:
prompt

"<s>[INST] You are an expert at extracting information from ocr-ed swedish land deeds sales data. You read and understand Swedish text given to you. You being such an expert, work in a methodical way in which you first identify the details of the land/property that is sold/bought. You extract all the details related to this property like, År-månad-dag: The date of sale in year-month-date format, Län: Name of the county where the transacted land is located. Härad: The precinct in which the transacted land is located, Nummer: The number(in the address) of the land that is transacted, By: The village in which the transacted land is located, Socken: The Parish in which the transacted land is located, Jordnatur: The type of the land that is transacted, Mantal: The number of Mantals of the land that is transacted, Örestal: The number of Örestals of the land that is transacted, Penningland: How many Penninglands of the land is transacted, Riksdaler: How many Riksdaler were transacted for the 

In [49]:
multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"
response_text = generate_response(multi_turn_prompt, temp=0.8, top_p=0.85)
print("Multi-turn conversation response:", response_text)

Multi-turn conversation response: GPT4 Correct System: Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpare-manlig: Namn på manliga köpare.

Köpare-kvinnlig: Namn på kvinnliga köpare.

K

In [50]:
multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"
response_text = generate_response(multi_turn_prompt, temp=0.2, top_p=0.1)
print("Multi-turn conversation response:", response_text)

Multi-turn conversation response: GPT4 Correct System: Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:

År-månad-dag: Identifiera datumet i formatet år-månad-dag.

Län: Namnet på länet.

Härad: Namnet på häradet.

Socken: Namnet på socknen.

By: Namnet på byn.

Jordnatur: Typ av jord eller mark.

Nummer: Numret på gården.

Mantal: Antal mantal.

Örestal: Antal örestal.

Penningland: Antal penningland.

Riksdaler: Antal riksdaler.

Öre: Antal öre.

Skilling: Antal skilling.

Runstycken: antal runstycken.

Valuta: Typ av valuta som används.

Säljare-manlig: Namn på manliga säljare.

Säljare-kvinnlig: Namn på kvinnliga säljare.

Köpare-manlig: Namn på manliga köpare.

Köpare-kvinnlig: Namn på kvinnliga köpare.

K

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 81.5 MB/s eta 0:00:00:00:01:01


In [24]:
df=pd.read_excel('Example database (1).xlsx')
df.head()

,Year-month-day,Län,Härad,Socken,Village,Jordnatur,Number,Mantal,Örestal,Penningland,...,Härad2,Socken2,Village2,Jordnatur2,Number2,Mantal2,Örestal2,Penningland2,image,comment
0,1860-11-17,Westmanland,Wagnsbro,Wester färnebo,Sör åll,Hemmanet,2,0.166667,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1861-02-26,Westmanlands,Wagnsbro,Wester fernebo,Islingsby,Skattehemmanet,5,0.133333,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1861-01-02,Westmanlands,Wagnsbro,Wester fernebo,Heden,skatte,NaN,0.125000,1.0,NaN,...,Wagnsbro,Wester fernebo,Skogsbyn,skatte,NaN,0.125,1.0,NaN,NaN,NaN
3,1861-01-19,Westmanlands,Wagnsbro,Karbennings,Labodarne,Hemman,NaN,0.031250,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1861-01-16,Westmanlands,Wagnsbro,Wester fernebo,Norr Sahlbo,Kronoskattehemmanet,4,0.041667,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_cols = df.columns

In [29]:
df_extracted_hightp_mix_np = pd.DataFrame(columns=df_cols)
df_extracted_hightp_mix_np

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Valuta,Säljare-manliga,Säljare-kvinnliga,Köpare-manliga,Köpare-kvinnliga,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion


In [68]:
df_cols.append('text')

In [37]:
df_cols

['År-månad-dag',
 'Län',
 'Härad',
 'Socken',
 'By',
 'Jordnatur',
 'Nummer',
 'Mantal',
 'Örestal',
 'Penningland',
 'Riksdaler',
 'Öre',
 'Skilling',
 'Runstycken',
 'Valuta',
 'Säljare-manlig',
 'Säljare-kvinnlig',
 'Köpare-manlig',
 'Köpare-kvinnlig',
 'Köpare-yrke',
 'Relation mellan säljare och köpare',
 'Köpare från',
 'Fördel',
 'Auktion',
 'text',
 'text']

In [ ]:
#new prompt
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+user_prompt},
]
prompt=tokenizer.apply_chat_template(messages, tokenize=False)

In [30]:

# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    # Construct the multi-turn prompt
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": pre_prompt+prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": pre_prompt+prompt4},
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": pre_prompt+text,
    },
    
    ]
    prompt=tokenizer.apply_chat_template(messages, tokenize=False)
    # Generate the response
    response_text = generate_response(prompt, temp=0.8, top_p=0.8)
    print()
    
    #print('{'+response_text.split('}')[-2].split('{')[-1]+'}')
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        #json_data = json.loads('{'+response_text.split('}')[-2].split('{')[-1]+'}')
        json_data = parse_nested_json(response_text)
        json_data['text'] = text
        #print(json_data)
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        json_data['text'] = text
        extracted_data.append(json_data)
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_hightp_mix_np = pd.DataFrame(extracted_data)


# Create a DataFrame from the collected data
df_extracted_hightp_mix_np = pd.DataFrame(extracted_data)
df_extracted_hightp_mix_np.to_csv('df_extracted_hightp_mix_np.csv')
df_extracted_hightp_mix_np.to_excel('df_extracted_hightp_mix_np.xlsx')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Row added!

Row added!


In [ ]:
#df_extracted_hightp_mix.to_csv('df_extracted_hightp_mix.csv')

In [ ]:
#df_extracted_hightp_mix.to_excel('df_extracted_hightp_mix.xlsx')

In [35]:
df_extracted_hightp_mix.columns

Index(['År-månad-dag', 'Län', 'Härad', 'Socken', 'By', 'Jordnatur', 'Nummer',
       'Mantal', 'Örestal', 'Penningland', 'Riksdaler', 'Öre', 'Skilling',
       'Runstycken', 'Valuta', 'Säljare-manlig', 'Säljare-kvinnlig',
       'Köpare-manlig', 'Köpare-kvinnlig', 'Köpare-yrke',
       'Relation mellan säljare och köpare', 'Köpare från', 'Fördel',
       'Auktion'],
      dtype='object')

In [34]:
df_extracted_hightp_mix

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Valuta,Säljare-manlig,Säljare-kvinnlig,Köpare-manlig,Köpare-kvinnlig,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion


In [74]:
pd.DataFrame(json.loads('{'+response_text.split('{')[-1]))

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Säljare-manliga,Säljare-kvinnliga,Köpare-manliga,Köpare-kvinnliga,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion,Vittnen
0,1860-11-17,Westmanland,Wagnsbro,Sura,Ståltorp,Hemmanet,2,1/6,1,8,...,Per Wrettström,Anna Persdotter,Johan Hedström,,,kan inte fastställas,,,,P. Pehrsson
1,1860-11-17,Westmanland,Wagnsbro,Sura,Ståltorp,Hemmanet,2,1/6,1,8,...,Per Wrettström,Anna Persdotter,Johan Hedström,,,kan inte fastställas,,,,Sör Gerdsbo
2,1860-11-17,Westmanland,Wagnsbro,Sura,Ståltorp,Hemmanet,2,1/6,1,8,...,Per Wrettström,Anna Persdotter,Johan Hedström,,,kan inte fastställas,,,,Lars Grönlund


In [76]:
'{'+response_text.split('{')[-1]

'{\n  "År-månad-dag": "1860-11-17",\n  "Län": "Westmanland",\n  "Härad": "Wagnsbro",\n  "Socken": "Sura",\n  "By": "Ståltorp",\n  "Jordnatur": "Hemmanet",\n  "Nummer": "2",\n  "Mantal": "1/6",\n  "Örestal": "1",\n  "Penningland": "8",\n  "Riksdaler": "1500",\n  "Öre": "",\n  "Skilling": "",\n  "Runstycken": "",\n  "Valuta": "Riksgels",\n  "Säljare-manliga": "Per Wrettström",\n  "Säljare-kvinnliga": "Anna Persdotter",\n  "Köpare-manliga": "Johan Hedström",\n  "Köpare-kvinnliga": "",\n  "Köpare-yrke": "",\n  "Relation mellan säljare och köpare": "kan inte fastställas",\n  "Köpare från": "",\n  "Fördel": "",\n  "Auktion": "",\n  "Vittnen": [\n    "P. Pehrsson",\n    "Sör Gerdsbo",\n    "Lars Grönlund"\n  ]\n}'

In [31]:
df_extracted_lowtp_mix_np = pd.DataFrame(columns=df_cols)
df_extracted_lowtp_mix_np

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Valuta,Säljare-manliga,Säljare-kvinnliga,Köpare-manliga,Köpare-kvinnliga,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion


In [32]:

# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    # Construct the multi-turn prompt
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": pre_prompt+prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": pre_prompt+prompt4},
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": pre_prompt+text,
    },
    
    ]
    prompt=tokenizer.apply_chat_template(messages, tokenize=False)
    # Generate the response
    response_text = generate_response(prompt, temp=0.2, top_p=0.2)
    print()
    
    #print('{'+response_text.split('}')[-2].split('{')[-1]+'}')
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        #json_data = json.loads('{'+response_text.split('}')[-2].split('{')[-1]+'}')
        json_data = parse_nested_json(response_text)
        json_data['text'] = text
        print(json_data)
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        json_data['text'] = text
        extracted_data.append(json_data)
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_lowtp_mix_np = pd.DataFrame(extracted_data)


# Create a DataFrame from the collected data
df_extracted_lowtp_mix_np = pd.DataFrame(extracted_data)
df_extracted_lowtp_mix_np.to_csv('df_extracted_lowtp_mix_np.csv')
df_extracted_lowtp_mix_np.to_excel('df_extracted_lowtp_mix_np.xlsx')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



{'År-månad-dag': '1860-11-17', 'Län': 'Westmanland', 'Härad': 'Wagnsbro', 'Socken': 'Wester färnebo', 'By': 'Sör åll', 'Jordnatur': 'Hemmanet', 'Nummer': '2', 'Mantal': '1/6', 'Örestal': '1', 'Penningland': '8', 'Riksdaler': '1500', 'Öre': '', 'Skilling': '', 'Runstycken': '', 'Valuta': 'Riksgels', 'Säljare-manliga': 'Per Wrettström', 'Säljare-kvinnliga': 'Anna Persdotter', 'Köpare-manliga': 'Johan Hedström', 'Köpare-kvinnliga': '', 'Köpare-yrke': '', 'Relation mellan säljare och köpare': 'Ej angiven', 'Köpare från': 'Ståltorp Sura Socken', 'Fördel': '', 'Auktion': '', 'text': '”Jag undertecknad uplåter och försäljer med min kära hustru ja och samtycke till Johan Hedström i Ståltorp Sura Socken min ägande 1/6 mantal Ett öre åtta penningland i Hemmanet no 2 i Sör åll af Wester färnebo Socken Wagnsbro härad och Westmanland län för en öfverenskommen och betingad köpeskilling stor Ett Tusende fem Hundra R Riksgels Sedlar och som jag denna köpeskilling till fullo bekommit hvilket och härme

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



{'Säljare': {'Name': 'Sara Jansdotter', 'Relation': 'Mother', 'Occupation': ''}, 'Köpare': {'Name': 'Johan Mattsson', 'Relation': 'Son', 'Occupation': 'Hemmansägaren'}, 'Property': {'Type': 'Skattehemmanet', 'Number': '5', 'Mantal': '2/15', 'Örestal': '', 'Penningland': '16', 'Address': {'Socken': 'Wester färnebo', 'By': 'Islingsby', 'Härad': 'Wagnsbro', 'Län': 'Westmanlands', 'Country': 'Sweden'}}, 'Köpeskilling': {'Amount': '709', 'Currency': 'Riksdaler', 'Öre': '33'}, 'Datum': '1861-02-26', 'Övrigt': '', 'Vittnen': [{'Name': 'J G Hild', 'Övrigt': 'E Borin'}], 'text': '”Jag Sara Jansdotter upplåter och försäljer härmedelst till min Son Hemmansägaren Johan Mattsson härstädes, det mig i arf efter min aflidne man Hemmansägaren Matts Jansson tillfallne två femton dels mantal af Skattehemmanet no 5, om Sexton penningeland i Islingsby Wester fernebo Socken, Wagnsbro Härad och Westmanlands Län belägen, emot en öfverenskommen och betingad Köpeskilling Stor: Sju Hundra Nio Riksdaler 33 öre R

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


{'År-månad-dag': '1853-06-17', 'Län': 'Nora', 'Härad': 'förene med Wagnsbro', 'Socken': 'Stalbo', 'By': 'uti förene meda By', 'Jordnatur': 'Krono skatte hemmanet', 'Nummer': '2', 'Mantal': '1/12', 'Örestal': '', 'Penningland': '12 8/12 dels', 'Riksdaler': '666', 'Öre': '32', 'Skilling': '', 'Runstycken': '', 'Valuta': 'Banko', 'Säljare-manliga': 'Anders Ersson', 'Säljare-kvinnliga': '', 'Köpare-manliga': 'Eric Ersson', 'Köpare-kvinnliga': 'Anna Ersdotter', 'Köpare-yrke': '', 'Relation mellan säljare och köpare': '', 'Köpare från': '', 'Fördel': '', 'Auktion': '', 'text': 'Jag Anders Ersson uti Stalbo Nora sn. uplåtter och för\nSellger mitt Egande 1/12 dels mantall om 12 8/12 dels pennin¬\ngeland uti Krono skatte hemmanet No 2 uti förennemda\nBy, ått Bonden Eric Ersson å hans hustru Anna Ers-\nsdoter i Berörde Stalbo för en öfverenskommen köppe\nskilling 666 Riksdaler 32 skl Banko och som jag\nDen första penningen med den sista till fullo nöije an-\nnammat hafver för den Skuld, så afhe

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



{'År-månad-dag': '1852-10-10', 'Län': 'Wesmanlands', 'Härad': 'Wåla', 'Socken': 'Wåhla', 'By': 'Bjurwalla', 'Jordnatur': 'Krono skatte', 'Nummer': '3', 'Mantal': '5 3/25', 'Örestal': '', 'Penningland': '', 'Riksdaler': '350', 'Öre': '', 'Skilling': '', 'Runstycken': '', 'Valuta': 'Riksgälds Sedlar', 'Säljare-manliga': 'Eric Olsson', 'Säljare-kvinnliga': 'Brita Ersdotters', 'Köpare-manliga': 'Olof Ersson', 'Köpare-kvinnliga': 'Anna Persdotter', 'Köpare-yrke': '', 'Relation mellan säljare och köpare': 'Bror och svägerska', 'Köpare från': 'Bjurwalla', 'Fördel': '', 'Auktion': '', 'text': 'Köpebref\nMed Sunt förnuft af fri vilja och moget\nbetänkande uplåter och försäljer jag Erik Olsson med\nmin kära hustru Brita Ersdotters goda ja och samtyc-\nke våre ägande i arf tillfallna fem (5 3/25)te dels pen-\ningelad i krono skatte No 3 Bjurwalla till wår broder\noch Svägerska Bonden Olof Ersson och dess hustru Anna\nPersdotter i Bjurwalla för en betingad och till fullo\nupburen köpeskilling sto

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



{'Köpare-manliga': 'Anders Mattsson', 'Köpare-kvinnliga': 'Brita Larsdotter', 'Köpare-yrke': '', 'Fördel': 'lifstids fördel', 'År-månad-dag': '1853-08-07', 'Län': 'Westmanlands', 'Härad': 'Wåla', 'Socken': 'Nora', 'By': 'Kjerstinbo', 'Jordnatur': 'kronoskatte', 'Nummer': '1/4', 'Mantal': '', 'Örestal': '', 'Penningland': '1 öres och 12 penningeland', 'Riksdaler': '2666', 'Öre': '32', 'Skilling': '', 'Runstycken': '', 'Valuta': 'Banko', 'Säljare-manliga': 'Matts Persson', 'Säljare-kvinnliga': 'Lisa Jönsdotter', 'Köpare från': '', 'text': 'Köpe Bref\nTill vår kära Son Anders Mattsson och dess blifvan-\nde hustru Brita Larsdotter i Kjerstinbo, uplåta och\nförsälja vi härigenom vårt ägande 1/4 Mantal kronoskatte\nom 1 öres och 12 penningeland uti Kjerstinbo, Nora Soc-\nken Wåla Härad och Westmanlands län beläget, för en \nöfverenskommen och betingad köpeskilling af Twå\ntusen Sexhundra Sextiosex Riksdaler och 32 sk Banko samt\nutfäst lifstids fördel; och som vi denna köpeskilliing\ntillfu

In [33]:
df_extracted_lowtp_mix_np.head()

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Övrigt,Vittnen,Övrig,Auctionsförrättare,Auctionplats,Auctiondatum,Sale Date,Auction Date,Price,Witnesses
0,1860-11-17,Westmanland,Wagnsbro,Wester färnebo,Sör åll,Hemmanet,2,1/6,1,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,"[{'Name': 'J G Hild', 'Övrigt': 'E Borin'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1861-01-02,Westmanlands,Wagnsbro,Wester färnebo,Heden,skatte,,1/8,1,1,...,NaN,NaN,Säljare försäljer 1/8 mantal skatte Heden och ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1861-01-19,Västmanlands,Wagnsbro,Karbennings,Klingholmen,Hemman,1/32,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1861-01-16,Westmanlands,Wagnsbro,Wester färnebo,Norr Sahlbo,Kronoskattehemmanet,4,1/24,,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": pre_prompt+prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": pre_prompt+prompt4},
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": pre_prompt+text,
    },
    
]
prompt=tokenizer.apply_chat_template(messages, tokenize=False)

In [34]:
df_extracted_lowthp_mix_np = pd.DataFrame(columns=df_cols)
df_extracted_lowthp_mix_np

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Valuta,Säljare-manliga,Säljare-kvinnliga,Köpare-manliga,Köpare-kvinnliga,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion


In [35]:

# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    # Construct the multi-turn prompt
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": pre_prompt+prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": pre_prompt+prompt4},
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": pre_prompt+text,
    },
    
    ]
    prompt=tokenizer.apply_chat_template(messages, tokenize=False)
    # Generate the response
    response_text = generate_response(prompt, temp=0.2, top_p=0.9)
    
    
    #print('{'+response_text.split('}')[-2].split('{')[-1]+'}')
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        #json_data = json.loads('{'+response_text.split('}')[-2].split('{')[-1]+'}')
        json_data = parse_nested_json(response_text)
        json_data['text'] = text
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        json_data['text'] = text
        extracted_data.append(json_data)
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_lowthp_mix_np = pd.DataFrame(extracted_data)





# Create a DataFrame from the collected data
df_extracted_lowthp_mix_np = pd.DataFrame(extracted_data)
df_extracted_lowthp_mix_np.to_csv('df_extracted_lowthp_mix_np.csv')
df_extracted_lowthp_mix_np.to_excel('df_extracted_lowthp_mix_np.xlsx')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!
Row added!


In [36]:
df_extracted_hightp_mix_np = pd.DataFrame(columns=df_cols)
df_extracted_hightp_mix_np

,År-månad-dag,Län,Härad,Socken,By,Jordnatur,Nummer,Mantal,Örestal,Penningland,...,Valuta,Säljare-manliga,Säljare-kvinnliga,Köpare-manliga,Köpare-kvinnliga,Köpare-yrke,Relation mellan säljare och köpare,Köpare från,Fördel,Auktion


In [38]:

# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    # Construct the multi-turn prompt
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
    {
        "role": "assistant",
        "content": sys_response1,
    },
    {"role": "user", "content": pre_prompt+prompt2},
    {
        "role": "assistant",
        "content": response2,
    },
    {"role": "user", "content": pre_prompt+prompt3},
    {
        "role": "assistant",
        "content": response3,
    },
    {"role": "user", "content": pre_prompt+prompt4},
    {
        "role": "assistant",
        "content": response4,
    },
    {
        "role": "user",
        "content": pre_prompt+text,
    },
    
    ]
    prompt=tokenizer.apply_chat_template(messages, tokenize=False)
    # Generate the response
    response_text = generate_response(prompt, temp=0.8, top_p=0.85)
    
    
    #print('{'+response_text.split('}')[-2].split('{')[-1]+'}')
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        #json_data = json.loads('{'+response_text.split('}')[-2].split('{')[-1]+'}')
        json_data = parse_nested_json(response_text)
        json_data['text'] = text
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        json_data['text'] = text
        extracted_data.append(json_data)
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_hightp_mix_np = pd.DataFrame(extracted_data)






# Create a DataFrame from the collected data
df_extracted_hightp_mix_np = pd.DataFrame(extracted_data)
df_extracted_hightp_mix_np.to_csv('df_extracted_hightp_mix_np.csv')
df_extracted_hightp_mix_np.to_excel('df_extracted_hightp_mix_np.xlsx')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row added!


TypeError: 'str' object does not support item assignment

In [8]:
prompt1 = "Hej!"
response1 = "Hallå! Hur kan jag hjälpa dig med att exakt extrahera information från svensk text i JSON-format?"
prompt2 =  user_prompt
response2 = """{
  "År-månad-dag": "1860-11-17",
  "Län": "Westmanland",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Sör åll",
  "Jordnatur": "Hemmanet",
  "Nummer": "2",
  "Mantal": "1/6",
  "Örestal": "1",
  "Penningland": "8",
  "Riksdaler": "1500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksgels",
  "Säljare-manliga": "Per Wrettström",
  "Säljare-kvinnliga": "Anna Persdotter",
  "Köpare-manliga": "Johan Hedström",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Ej angiven",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt3="""Extrahera informationen för varje fält och presentera den tydligt i JSON-format. 'Jag Sara Jansdotter upplåter och försäljer härmedelst till min Son Hemmansägaren Johan Mattsson härstädes, det mig i arf efter min aflidne man Hemmansägaren Matts Jansson tillfallne två femton dels mantal af Skattehemmanet no 5, om Sexton penningeland i Islingsby Wester fernebo Socken, Wagnsbro Härad och Westmanlands Län belägen, emot en öfverenskommen och betingad Köpeskilling Stor: Sju Hundra Nio Riksdaler 33 öre Riksmynt, och som jag denna köpeskilling tillfullo bekommit, alltså afhänder jag mig och mina öfrige arfvingar berörde 2/15 dels mantal Islingsby, med allt hvad dertill hörer, af ålder lydt och lagligen tillvinnas kan, och tillegne den samma bemälde min Son Johan Mattsson och hans arfvingar, att genast tillträda, samt everldeliga äga och besitta hafvande till yttermera wisse häraf, jag detta köpebref uti nedanstående wittnens närvaro, med mitt bomärkes undertecknande bekräfta, som skedde i Islingsby Nybygget den 26 februari 1861 Sara/bom/Jansdotter wittnen underskrifna'"""
response3="""{
  "År-månad-dag": "1861-02-26",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Islingsby",
  "Jordnatur": "Skattehemmanet",
  "Nummer": "5",
  "Mantal": "2/15",
  "Örestal": "",
  "Penningland": "16",
  "Riksdaler": "709",
  "Öre": "33",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Sara Jansdotter",
  "Säljare-kvinnliga": "",
  "Köpare-manliga": "Johan Mattsson",
  "Köpare-kvinnliga": "",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Mor och Son",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
prompt4="""'Jag Eric Jansson gör hervid veterligt att jag med min kära hustru Anna Jansdotters goda ja och samtycke samt af fri vilja, försålt, såsom jag ock genom detta öppna salubref, försäljer till min Svärfar Hemmansegaren Jan Ersson och hustru Anna Jansdotter, egona 1/8 mtl skatte Heden om 1 öresland och 1/8 mtl skatte Skogsbyn om 1 öresland i Wester fernebo Socken och Wagnsbro härad för en Summa af Sjutusen femhundra /7500/ Riksdaler Riksmynt och som den … penningen med den första är till mig slutligen betald så afhändar jag mig, min köra hustru och barn tillhörde hemmansdelar 1/8 mlt Heden och 1/8 mtl Skogby med alla dess tillhörigheter och tillägnar de samma Jan Ersson och hustru Anna Jansdotter, Heden och arfvingar med hemmansdelar med alla ägor i hus, jord och öfrige tillhörigheter intet undantagandes af hvad dertill lyder af ålder legat eller framledes tillvinnas kan, att äga och bruka och besitta såsom deras välfångna egendom, förbindande till hemul efter lag – Till yttermera visshet har jag tillika med min kära hustru detta Salubref i undertecknande vittnens närvaro, egenhändigt underskrifvet som skedde på Heden den 2ra januari 1861. Eric Jansson (otydligt) och Heden. Anna /bom/ kattharina Jansdotter
Jan Ersson Boende på Heden. Anders Mattsson Hebo egor'"""
response4= """{
  "År-månad-dag": "1861-01-02",
  "Län": "Westmanlands",
  "Härad": "Wagnsbro",
  "Socken": "Wester färnebo",
  "By": "Heden",
  "Jordnatur": "Skatte",
  "Nummer": "",
  "Mantal": "1/8",
  "Örestal": "1",
  "Penningland": "",
  "Riksdaler": "7500",
  "Öre": "",
  "Skilling": "",
  "Runstycken": "",
  "Valuta": "Riksmynt",
  "Säljare-manliga": "Eric Jansson",
  "Säljare-kvinnliga": "Anna Jansdotters",
  "Köpare-manliga": "Jan Ersson",
  "Köpare-kvinnliga": "Anna Jansdotter",
  "Köpare-yrke": "",
  "Relation mellan säljare och köpare": "Svärfar och Svärson",
  "Köpare från": "",
  "Fördel": "",
  "Auktion": ""
}"""
#prompt5="Jag Per Gustaf Hedberg tillika med min hustrus Christina Carolina Gerdtsdotter samtycke upplåter och försäljer till Per Gustaf Ersson och dess hustru Lovisa Ersdotter Klingholmen vårt uti Labodarne ägande Ett Trettiondetvånde dels Hemman med dertill hörande inägojord skog och mark beläget inom Karbennings Socken dock likväl Wagnsbro Härad emot en överenskommen och betingad köpeskilling stor Ett Tusen Två Hundrade Riksdaler Riksmynt, Och som denna köpeskilling 1200 Rd Rmt är uti ett särskildt köpekontrakt häröfver af denna dag upptagen så varder den derigenom här qvitterad. Alltså afhänder vi oss och våra efter-kommande berörde 1/32de dels Hemman med åtföljande förmoner samt alt hvad hädanefter lagligen tillvinnas kan till bemälte Per Gustaf Ersson dess hustru och efterkommande rätts innehafvare att everlderligen äga bruka och besitta, förbindande vi oss till Himmel efter lag hvadan vi uti tvenne tillkallade wittnens när och öfvervaro underskrifne som skedde uti Karbenningby den 19 januari 1861 Per Gustaf Hedberg Christina Carolina Gerdtsdotter i Karbenningby Wittnar Lars Eric Larsson Lars Starksson."

In [ ]:
multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"
response_text = generate_response(multi_turn_prompt, temp=0.2, top_p=0.1)
print("Multi-turn conversation response:", response_text)

In [ ]:
# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    text='Extrahera noggrant informationen för varje fält och presentera den tydligt i JSON-format. dubbelkolla noggrannheten för varje extraherat fält innan presentationen' + "'''"+text.strip()+"'''"
    # Construct the multi-turn prompt
    multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"

    # Generate the response
    response_text = generate_response(multi_turn_prompt, temp=0.8, top_p=0.85)
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        json_data = json.loads('{'+response_text.split('{')[-1])
        json_data['text'] = text
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        json_data['text'] = text
        extracted_data.append(json_data)
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_hightp = pd.DataFrame(extracted_data)
df_extracted_hightp.to_csv('Extracted_hightnp.csv')
df_extracted_hightp.to_excel('Extracted_hightnp.xlsx')

In [13]:
#chain of thought user prompt test 1
# Initialize a list to collect data
extracted_data = []

# Loop through each text in the DataFrame
for text in df['Text']:
    text='Tänk noga steg för steg och extrahera informationen för varje fält i texten från ett markförsäljningskontrakt och presentera det tydligt i JSON-format. dubbelkolla noggrannheten för varje extraherat fält innan presentationen' + "'''"+text.strip()+"'''"
    # Construct the multi-turn prompt
    multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {text}<|end_of_turn|>GPT4 Correct Assistant:"
    #multi_turn_prompt = f"GPT4 Correct System: {system_prompt}<|end_of_turn|>GPT4 Correct User: {prompt1}<|end_of_turn|>GPT4 Correct Assistant: {response1}<|end_of_turn|>GPT4 Correct User: {prompt2}<|end_of_turn|>GPT4 Correct Assistant: {response2}<|end_of_turn|>GPT4 Correct User: {prompt3}<|end_of_turn|>GPT4 Correct Assistant: {response3}<|end_of_turn|>GPT4 Correct User: {prompt4}<|end_of_turn|>GPT4 Correct Assistant:{response4}<|end_of_turn|>GPT4 Correct User: {prompt5}<|end_of_turn|>GPT4 Correct Assistant:"

    # Generate the response
    response_text = generate_response(multi_turn_prompt, temp=0.8, top_p=0.85)
    
    # Parse JSON and add to the list
    try:
        # Extract JSON from the response
        json_data = json.loads('{'+response_text.split('{')[-1])
        extracted_data.append(json_data)
        print('Row added!')
    except json.JSONDecodeError:
        print("Error parsing JSON for response: ", response_text)

# Create a DataFrame from the collected data
df_extracted_hightp = pd.DataFrame(extracted_data)
df_extracted_hightp.to_csv('COTExtracted_hightnp.csv')
df_extracted_hightp.to_excel('COTExtracted_hightnp.xlsx')


Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Error parsing JSON for response:  GPT4 Correct System: Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:
År-månad-dag: Identifiera datumet i formatet år-månad-dag.
Län: Namnet på länet.
Härad: Namnet på häradet.
Socken: Namnet på socknen.
By: Namnet på byn.
Jordnatur: Typ av jord eller mark.
Mantal: Information om mantal.
Örestal: Antal öre.
Penningland: Information om penningland.
Riksdaler: Antal riksdaler.
Öre: Antal öre.
Skilling: Antal skilling.
Runstycken: Information om runstycken.
Valuta: Typ av valuta som används.
Sälja

Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Error parsing JSON for response:  GPT4 Correct System: Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:
År-månad-dag: Identifiera datumet i formatet år-månad-dag.
Län: Namnet på länet.
Härad: Namnet på häradet.
Socken: Namnet på socknen.
By: Namnet på byn.
Jordnatur: Typ av jord eller mark.
Mantal: Information om mantal.
Örestal: Antal öre.
Penningland: Information om penningland.
Riksdaler: Antal riksdaler.
Öre: Antal öre.
Skilling: Antal skilling.
Runstycken: Information om runstycken.
Valuta: Typ av valuta som används.
Säljare-manlig: Namn på manliga säljare.
Säljare-kvinnlig: Namn på kvinnliga säljare.
Köpare-manlig: Namn på manlig

Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Error parsing JSON for response:  GPT4 Correct System: Hej, jag behöver din hjälp för att extrahera specifik information från en svensk text. Var noggrann och ge endast information som finns tydligt i texten. Om du är osäker på något, eller om informationen inte finns i texten, är det bättre att inte ge ett svar för den specifika punkten. Här är de fält jag behöver information om:
År-månad-dag: Identifiera datumet i formatet år-månad-dag.
Län: Namnet på länet.
Härad: Namnet på häradet.
Socken: Namnet på socknen.
By: Namnet på byn.
Jordnatur: Typ av jord eller mark.
Mantal: Information om mantal.
Örestal: Antal öre.
Penningland: Information om penningland.
Riksdaler: Antal riksdaler.
Öre: Antal öre.
Skilling: Antal skilling.
Runstycken: Information om runstycken.
Valuta: Typ av valuta som används.
Säljare-manlig: Namn på manliga säljare.
Säljare-kvinnlig: Namn på kvinnliga säljare.
Köpare-manlig: Namn på manliga köpare.
K

Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
Row added!
